In [1]:
import os
%pwd

'f:\\ml projects\\brain_tumor_classification\\notebooks'

In [2]:
os.chdir("../")
%pwd

'f:\\ml projects\\brain_tumor_classification'

In [3]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class DataIngestionConfig:
    root_dir:Path
    source: str
    local_data_file:Path
    unzip_dir:Path

In [4]:
from brain_tumor_classification.constants import *
from brain_tumor_classification.utils.common import create_directories,read_yaml

class ConfigurationManager:
    def __init__(self,config_filepath = CONFIG_FILEPATH,params_filepath = PARAMS_FILEPATH):
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)

        create_directories([self.config.artifacts_root])

    def get_data_ingestion_config(self) -> DataIngestionConfig:
        
        temp_config = self.config.data_ingestion

        create_directories([temp_config.root_dir])

        data_ingestion_config = DataIngestionConfig(
            root_dir = temp_config.root_dir,
            source = temp_config.source,
            local_data_file = temp_config.local_data_file,
            unzip_dir = temp_config.unzip_dir 
        )

        return data_ingestion_config
        


In [5]:
import os,sys
from urllib import request
import zipfile
from brain_tumor_classification.utils.logger import logger
from brain_tumor_classification.utils.common import get_size
from brain_tumor_classification.utils.exception import CustomException
import gdown

In [6]:
class DataIngestion:
    def __init__(self,config:DataIngestionConfig):
        self.config = config

    def download_file(self):
        if not os.path.exists(self.config.local_data_file):
            gdown.download(self.config.source,self.config.local_data_file,quiet=False)
            logger.info(f'{self.config.local_data_file} Downloaded')
        else:
            logger.info(f'File Already Exists: {get_size(Path(self.config.local_data_file))}')

    def extract_file(self):
        unzip_path = self.config.unzip_dir
        with zipfile.ZipFile(self.config.local_data_file,'r') as zip_ref:
            zip_ref.extractall(unzip_path)

In [7]:
try:
    config = ConfigurationManager()
    data_ingestion_config = config.get_data_ingestion_config()
    data_ingestion = DataIngestion(data_ingestion_config)
    data_ingestion.download_file()
    data_ingestion.extract_file()
except Exception as e:
    raise CustomException(e,sys)
    

Downloading...
From (uriginal): https://drive.google.com/uc?id=1pNuyMlW0FkbP-0C08RE5MePLY6YbUxBo
From (redirected): https://drive.google.com/uc?id=1pNuyMlW0FkbP-0C08RE5MePLY6YbUxBo&confirm=t&uuid=bfa3b3a9-4871-4916-ae59-d5d8d8252382
To: f:\ml projects\brain_tumor_classification\artifacts\data_ingestion\data.zip
100%|██████████| 156M/156M [00:23<00:00, 6.74MB/s] 

[ 2023-11-25 21:28:03,438 ] - 8 - 1520497564 - artifacts/data_ingestion/data.zip Downloaded
